<a href="https://colab.research.google.com/github/Shopping-Yuan/ML2021HW/blob/Shopping_branch/ML2021HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
To: /content/data.zip
100% 372M/372M [00:04<00:00, 84.5MB/s]
Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  sample_data  timit_11


In [2]:
#import pytorch
import torch

# torch.backends.cudnn: set CNN algorithmtorch.backends.cudnn
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# get the current available device ('cpu' or 'cuda')
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(torch.cuda.is_available())
#set random variable
import numpy as np
myseed = 1
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

True


In [3]:
#write data_set_function
#normalize data if needed
import pandas as pd
#Dataset:data_set_function
import numpy as np

def TIMIT_train_val_f(mode,d_l = "both"):

  data = np.load(data_info[mode]["data_path"])
  label = np.load(data_info[mode]["label_path"])
  print('Size of training data: {}'.format(data.shape))
  print('Size of training label: {}'.format(label.shape))
  VAL_RATIO = 0.2
  percent = int(data.shape[0] * (1 - VAL_RATIO))
  train_data, train_label, val_data, val_label = \
  data[:percent], label[:percent].astype(np.int_), data[percent:], label[percent:].astype(np.int_)

  if mode == "train":
    return{"data" : torch.FloatTensor(train_data) , "label" : torch.LongTensor(train_label)}
  elif mode == "val":
    return{"data" : torch.FloatTensor(val_data) , "label" : torch.LongTensor(val_label)}

def TIMIT_test_f(mode,d_l = "data"):
  test = np.load(data_info[mode]["path"])
  print('Size of testing data: {}'.format(test.shape))
  return torch.FloatTensor(test)

#create a dict of functions and path w.r.t. different mode
data_info = {
    "train":{"function":TIMIT_train_val_f,"data_path":"./timit_11/train_11.npy","label_path":"./timit_11/train_label_11.npy","data_or_label":"both"},
    "val":{"function":TIMIT_train_val_f,"data_path":"./timit_11/train_11.npy","label_path":"./timit_11/train_label_11.npy","data_or_label":"both"},
    "test":{"function":TIMIT_test_f,"path":"./timit_11/test_11.npy","data_or_label":"data"}
}

In [4]:
#rewrite class Dataset
from torch.utils.data import Dataset, DataLoader
class Dataset_preparation(Dataset):
  def __init__(self,mode,data_info):
      self.mode = mode
      self.data_or_label = data_info[mode]["data_or_label"]

      if self.data_or_label == "data":
        self.data = data_info[self.mode]["function"](self.mode,"data")
      elif self.data_or_label == "both":
        data_and_label = data_info[self.mode]["function"](self.mode,"both")
        self.data = data_and_label["data"]
        self.label = data_and_label["label"]
      else :
        self.label = data_info[self.mode]["function"](self.mode,"label")["label"]

      self.dim = self.data.shape[1]
      print('Finished reading the {mode} set of Dataset ({len} samples found, each dim = {dim})'
              .format(mode = self.mode, len =len(self.data), dim=self.dim))

  def __getitem__(self, index):
      if self.data_or_label == "data":
        return self.data[index]
      elif self.data_or_label == "both":
        return self.data[index], self.label[index]
      else :
        return self.label[index]
  def __len__(self):
      # Returns the size of the dataset
      return len(self.data)

In [5]:
from torch.utils.data import DataLoader
#decide how to load data
def prep_dataloader(mode,d_info,batch_size,n_jobs=0):
    dataset = Dataset_preparation(mode,d_info)
    dataloader = DataLoader(
        dataset, batch_size,
        shuffle=(mode == 'train'), drop_last=False,
        num_workers=n_jobs, pin_memory=False)
    return dataloader

In [22]:
#Model : class Classifier
import torch.nn as nn
class Classifier(nn.Module):
    def __init__(self, input_dim):
        super(Classifier, self).__init__()

        self.net = nn.Sequential(
              nn.Linear(input_dim, 1024),
              nn.LeakyReLU(),
              nn.Linear(1024, 512),
              nn.LeakyReLU(),
              nn.Linear(512, 256),
              nn.LeakyReLU(),
              nn.Linear(256, 39),
              )
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.net(x)
        return x

    def cal_loss(self, pred, target):
        return self.criterion(pred, target)

In [16]:
h_paras = {
    # maximum number of epochs
    'n_epochs': 200,
    # mini-batch size for dataloader
    'batch_size': 640,
    # optimization algorithm (optimizer in torch.optim)
    'optimizer': 'Adam',
    # hyper-parameters for the optimizer (depends on which optimizer you are using)
    'optim_hparas': {
        # learning rate of Adam
        'lr': 0.001,
    },
    'early_stop': 10,
    # your model will be saved here
    'save_path': './model.pth'
}

In [8]:
train_set = prep_dataloader("train",data_info,h_paras['batch_size'])
val_set = prep_dataloader("val",data_info,h_paras['batch_size'])
test_set = prep_dataloader("test",data_info,h_paras['batch_size'])

Size of training data: (1229932, 429)
Size of training label: (1229932,)
Finished reading the train set of Dataset (983945 samples found, each dim = 429)
Size of training data: (1229932, 429)
Size of training label: (1229932,)
Finished reading the val set of Dataset (245987 samples found, each dim = 429)
Size of testing data: (451552, 429)
Finished reading the test set of Dataset (451552 samples found, each dim = 429)


In [9]:
#print(TIMIT_train_val_f("train",d_l = "both")["data"][:5,:5])

In [10]:
def train(train_set,model_d,optimizer,device):
    # set model to training mode
    model_d.train()
    total_correct_number = 0
    train_loss_list = []
    # iterate through the dataloader
    for data , label in train_set:
      # move data to device (cpu/cuda)
      data_d , label_d = data.to(device), label.to(device)
      # forward pass (compute output tensor)
      pred = (model_d(data_d))
      # get the index of the class with the highest probability
      max_prob_values, max_prob_indexs = torch.max(pred, dim = 1)
      correct_number = (max_prob_indexs.cpu() == label_d.cpu()).sum().item()
#      print(correct_number)
      total_correct_number += correct_number
      # compute loss
      loss = model_d.cal_loss(pred , label_d)
      # compute gradient (backpropagation)
      loss.backward()
      # update model with optimizer
      optimizer.step()
      # set optimizer gradient to zero
      optimizer.zero_grad()
      train_loss_list.append(loss.detach().cpu().item())
    acc = total_correct_number/len(train_set.dataset)
    return acc , train_loss_list



In [11]:
#a = torch.tensor([[0,1],[2,3]])
#b = torch.tensor([[4,1],[5,3]])
#print(type(a == b))

In [12]:
def val(val_set,model_d,device):
    # set model to evalutation mode
    model_d.eval()
    total_correct_number = 0
    total_loss = 0
    # iterate through the dataloader
    for data , label in val_set:
    # move data to device (cpu/cuda)
      data_d, label_d = data.to(device), label.to(device)
      # disable gradient calculation
      with torch.no_grad():
        # forward pass (compute output)
        pred = model_d(data_d)
        # get the index of the class with the highest probability
        max_prob_values, max_prob_indexs = torch.max(pred, dim = 1)
        total_correct_number += (max_prob_indexs.cpu() == label_d.cpu()).sum().item()
        # compute loss
        mse_loss = model_d.cal_loss(pred, label_d)
      # accumulate loss
      batch_size = len(data_d)
      total_loss += mse_loss.detach().cpu().item() * batch_size

    # compute averaged loss
    totol_size = len(val_set.dataset)
    acc = total_correct_number/totol_size
    avg_loss =  total_loss/totol_size
    return acc, avg_loss

In [61]:
def test(test_set, model_d, device):
    # set model to evalutation mode
    model_d.eval()
    pred_acc = []
    pred_loss = []
    # iterate through the dataloader
    for data in test_set:
      # move data to device (cpu/cuda)
      data_d = data.to(device)
      # disable gradient calculation
      with torch.no_grad():
        # forward pass (compute output)
        pred = model_d(data_d)
        # get the index of the class with the highest probability
        max_prob_values, max_prob_indexs = torch.max(pred, dim = 1)
        # collect prediction
        pred_acc += (max_prob_indexs.tolist())
        pred_loss += (pred.detach().tolist())
    # concatenate all predictions and convert to a numpy array
    np_pred_acc , np_pred_loss = np.array(pred_acc), np.array(pred_loss).mean(axis = 1)
    pred_dict = { "pred_acc" : np_pred_acc , "pred_loss" : np_pred_loss }
    return pred_dict



In [23]:
def train_val_process(training_set, validation_set, model, h_paras, device):
    #1: move model to device (cpu/cuda)
    model_d = model.to(device)
    #2: set optimizer = torch.optim.SGD(model_d.parameters(),lr=0.001,momentum=0.9)
    lr_now = h_paras['optim_hparas']["lr"]
    optimizer = getattr(torch.optim, h_paras['optimizer'])(
        model_d.parameters(), lr = lr_now)

    #3: set epoch = h_paras['n_epochs'] = 3000
    n_epochs = h_paras['n_epochs']

    #4: record training accuracy
    acc_record = {'train': [], "val": []}
    #5: record training loss
    loss_record = {'train': [], "val": []}

    #6: setting paras
    # epoch para
    epoch = 0
    # accuracy paras
    best_acc = 0
    # early-stoping paras
    early_stop_cnt = 0
    #start training
    while epoch < n_epochs:

      # Part1 : training process
      # updata model weight and compute model loss
      train_acc , train_loss_list = train(training_set,model_d,optimizer,device)
      # save accuracy to acc_record['train']
      acc_record['train'].append(train_acc)
      # save loss to loss_record['train']
      loss_record['train'].append(train_loss_list)
      # renew epoch para
      epoch += 1
      print('train set accuracy = {:.3f}'.format(train_acc))
      # Part2 validation process
      # After each epoch, test your model
      # on the validation (development) set.
      val_acc , val_loss = val(validation_set, model_d, device)
      # save accuracy to acc_record["val"]
      acc_record["val"].append(val_acc)
      # save loss to loss_record["val"]
      loss_record["val"].append(val_loss)

      print(' model epoch = {:4d}, train_loss = {:.4f} , val_loss = {:.4f})'\
      .format(epoch , train_loss_list[-1] , val_loss))
      # Save model to specified path if your model improved
      torch.save(model_d.state_dict(), h_paras['save_path'])
      if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), h_paras["save_path"])
        print('saving model with acc {:.3f}'.format(best_acc))
        early_stop_cnt = 0
      else:
        early_stop_cnt += 1
      # Check early stop criteria
      if early_stop_cnt > h_paras['early_stop']:
          # Stop training if your model stops improving
          # for "h_paras['early_stop']" epochs.
          break
      elif ((early_stop_cnt > 1) and (lr_now>0.0001)):
        lr_now = lr_now * 0.5
        optimizer = getattr(torch.optim, h_paras['optimizer'])(
        model_d.parameters(), lr = lr_now)
        n_epochs += 20
#        early_stop_cnt = 0

    print('Finished training after {} epochs'.format(epoch))
    return acc_record , loss_record

In [24]:
# Construct model
model = Classifier(train_set.dataset.dim)
model_loss, model_loss_record = train_val_process(train_set, val_set, model, h_paras, device)

train set accuracy = 0.644
 model epoch =    1, train_loss = 0.7550 , val_loss = 0.9646)
saving model with acc 0.689
train set accuracy = 0.717
 model epoch =    2, train_loss = 0.8800 , val_loss = 0.8986)
saving model with acc 0.709
train set accuracy = 0.746
 model epoch =    3, train_loss = 0.6760 , val_loss = 0.8991)
saving model with acc 0.713
train set accuracy = 0.768
 model epoch =    4, train_loss = 0.6548 , val_loss = 0.9259)
train set accuracy = 0.787
 model epoch =    5, train_loss = 0.5409 , val_loss = 0.9565)
train set accuracy = 0.833
 model epoch =    6, train_loss = 0.5760 , val_loss = 0.9818)
saving model with acc 0.715
train set accuracy = 0.851
 model epoch =    7, train_loss = 0.4151 , val_loss = 1.0433)
train set accuracy = 0.865
 model epoch =    8, train_loss = 0.3218 , val_loss = 1.1100)
train set accuracy = 0.896
 model epoch =    9, train_loss = 0.3345 , val_loss = 1.1685)
train set accuracy = 0.917
 model epoch =   10, train_loss = 0.3318 , val_loss = 1.2279

In [75]:
# # start training
# best_acc = 0.0
# for epoch in range(num_epoch):
#     train_acc = 0.0
#     train_loss = 0.0
#     val_acc = 0.0
#     val_loss = 0.0

#     # training
#     model.train() # set the model to training mode
#     for i, data in enumerate(train_loader):
#         inputs, labels = data
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         batch_loss = model.cal_loss(outputs, labels)
#         _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
#         batch_loss.backward()
#         optimizer.step()

#         train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
#         train_loss += batch_loss.item()

#     # validation
#     if len(val_set) > 0:
#         model.eval() # set the model to evaluation mode
#         with torch.no_grad():
#             for i, data in enumerate(val_loader):
#                 inputs, labels = data
#                 inputs, labels = inputs.to(device), labels.to(device)
#                 outputs = model(inputs)
#                 batch_loss = model.cal_loss(outputs, labels)
#                 _, val_pred = torch.max(outputs, 1)

#                 val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
#                 val_loss += batch_loss.item()

#             print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
#                 epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
#             ))

#             # if the model improves, save a checkpoint at this epoch
#             if val_acc > best_acc:
#                 best_acc = val_acc
#                 torch.save(model.state_dict(), model_path)
#                 print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
#     else:
#         print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
#             epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
#         ))

# # if not validating, save the last epoch
# if len(val_set) == 0:
#     torch.save(model.state_dict(), model_path)
#     print('saving model at last epoch')

In [73]:
import pandas as pd
def save_pred(pred_dict, file_path):
    print('Saving results to {}'.format(file_path))
    acc = pred_dict["pred_acc"]
    loss = pred_dict["pred_loss"]
    print(acc.shape,loss.shape)
    for  index ,(a , l) in enumerate(zip(acc,loss)):
      if index%1000 == 0:
        print(a , l)
    df = pd.DataFrame({"acc": acc, "loss": loss})
    df.to_csv(file_path)

In [63]:
pred_dict = test(test_set, model, device)  # predict cases with your model

In [72]:
save_pred(pred_dict, 'pred.csv')         # save prediction file to pred.csv
#del model

串流輸出內容已截斷至最後 5000 行。
19 -10.204852054516474
19 -7.862689588314447
19 -6.636612486380797
38 -5.48205876426819
22 -5.771793737243383
22 -8.99536574498201
22 -12.646488424295034
22 -13.981406334118965
22 -12.666311874603613
1 -10.856151524644632
1 -10.382456457003569
1 -10.283673767095957
1 -10.622912529187325
1 -10.393515950594193
1 -10.809312655375553
1 -13.354949242411516
19 -13.85079098531069
5 -12.812240689228743
1 -13.387433155989035
1 -13.332884137447063
1 -11.876438481685442
1 -12.290297429531048
1 -11.50829421938994
1 -10.358616028076563
31 -8.635374008080898
31 -8.698235755165419
31 -8.648098539847593
35 -9.805099268372242
35 -10.154293747284473
35 -10.182439107161303
35 -8.904794882505367
31 -7.370332941030845
31 -7.775591838245209
31 -7.926064947285713
35 -8.289379649437391
35 -6.956192814386808
35 -7.791156350037991
35 -8.960352598092495
35 -9.317424703867008
35 -8.80392417540917
35 -8.45480061494387
36 -7.130078699344244
36 -7.997820895451766
36 -11.825768245336336
36 -13.57

KeyboardInterrupt: ignored

In [65]:
# create model and load weights from checkpoint
model = Classifier(train_set.dataset.dim).to(device)
model_loading = torch.load(h_paras['save_path'], map_location=device)
model.load_state_dict(model_loading,strict=True)

<All keys matched successfully>

In [69]:
#with open('prediction.csv', 'w') as f:
#    f.write('Id,Class\n')
#for acc , loss in (test(test_set, model, device).values(),test(test_set, model, device).values()):
#        f.write('{},{}\n'.format(n_epoch, acc,loss))


In [ ]:
import matplotlib.pyplot as plt
def plot_learning_curve(loss_record, title=''):
    total_steps = len(loss_record['train'])
    x_1 = range(total_steps)
    x_2 = x_1[::len(loss_record['train']) // len(loss_record['val'])]
    plt.figure(figsize=(6, 4))
    plt.plot(x_1, loss_record['train'], c='tab:red', label='train')
    plt.plot(x_2, loss_record['val'], c='tab:cyan', label='val')
    plt.ylim(0.0, 10.)
    plt.xlabel('Training steps')
    plt.ylabel('MSE loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()


def plot_pred(val_set, model, device, lim=50., preds=None, targets=None):
    if preds is None or targets is None:
        model.eval()
        preds, labels = [], []
        for data , label in val_set:
            data_d , label_d = data.to(device), label.to(device)
            with torch.no_grad():
                pred = model(data_d)
                preds += pred.detach().tolist()
                labels += label_d.detach().tolist()
        preds = np.array(preds)
        labels = np.array(labels)
    plt.figure(figsize=(5, 5))
    plt.scatter(labels, preds, c='r', alpha=0.5)
    plt.plot([-0.2, lim], [-0.2, lim], c='b')
    plt.xlim(-0.2, lim)
    plt.ylim(-0.2, lim)
    plt.xlabel('ground truth value')
    plt.ylabel('predicted value')
    plt.title('Ground Truth v.s. Prediction')
    plt.show()

In [ ]:
plot_learning_curve(model_loss_record, title='deep model')
plot_pred(val_set, model, device)  # Show prediction on the validation set